In [8]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor


allMerged=pd.read_csv("merged.csv")
weather = pd.read_csv("weather.csv")
holidays = pd.read_csv("holidays.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")
test = pd.read_csv("test.csv")

def generate_value(date):
    # Yıl, ay ve gün bilgilerini ayırma
    year, month, day = map(int, date.split('-'))
    
    # Ay bilgisini 1 ile, gün bilgisini 2 ile çarpma
    value = month * 1 + day * 2
    
    return value

# Fonksiyonu DataFrame'deki tarih sütununa uygulama
allMerged['tarih'] = allMerged['tarih'].apply(lambda x: generate_value(x))

print(allMerged.info())
# Bayram_Flag sütunundaki birden fazla bayramı içeren girişleri işleme
holidays["Tatil Adı"] = holidays["Tatil Adı"].apply(lambda x: x.split(';')[0] if ';' in x else x)

# 'Tatil Adı' sütunundaki birden fazla bayramı içeren girişleri işleme
holidays["Tatil Adı"] = holidays["Tatil Adı"].str.split(';').explode()

# 'Yılbaşı' değerini '1', diğer değerleri '0' olarak kodlayan bir fonksiyon
def encode_holiday_flag(value):
    if value == 'Yılbaşı':
        return 1
    else:
        return 0

# 'Tatil Adı' sütununu kodlayın
holidays['Tatil Adı'] = holidays['Tatil Adı'].apply(encode_holiday_flag)

def convert_to_year_month_day(date):
    # datetime nesnesini oluşturma
    dt = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    
    # Yıl, ay ve gün bilgilerini alma ve birleştirme
    year_month_day = dt.strftime("%Y-%m-%d")
    
    return year_month_day
weather['date'] = weather['date'].apply(convert_to_year_month_day)
# Yıl, Ay ve Gün sütunlarını tarih formatına dönüştürme
holidays["Gün"] = pd.to_datetime(holidays[["Yıl", "Ay", "Gün"]].astype(str).agg('-'.join, axis=1))
weather["date"] = pd.to_datetime(weather["date"]) # Tarih sütununu datetime türüne dönüştürme
test["tarih"] = pd.to_datetime(test["tarih"])
test = pd.merge(test, weather, left_on=["tarih", "ilce"], right_on=["date", "name"], how="left")
test = pd.merge(test, holidays, left_on=["tarih"], right_on=["Gün"], how="left")
print(test.head())
# Özellikleri seçme
features = ["tarih","ilce","bildirimli_sum","t_2m:C", "effective_cloud_cover:p", "global_rad:W", "relative_humidity_2m:p",
            "wind_dir_10m:d", "wind_speed_10m:ms", "prob_precip_1h:p", "t_apparent:C"]
# Label encoding için LabelEncoder kullanma
label_encoder = LabelEncoder()

# Eğitim ve test setlerini oluşturma
X_train = allMerged[features]
X_train['ilce'] = label_encoder.fit_transform(X_train['ilce'])
y_train = allMerged["bildirimsiz_sum"]
X_test = test[features]
X_test['ilce'] = label_encoder.fit_transform(X_test['ilce'])
print(X_test.head())
def convert_to_year_month_day(date):
    # Tarihi stringe dönüştürme
    date_str = str(date)
    
    # Tarihi parçalara ayırma
    parts = date_str.split()[0].split("-")
    
    # Yıl, ay ve gün bilgilerini alma ve birleştirme
    year_month_day = "-".join(parts)
    
    return year_month_day
X_test['tarih'] =X_test['tarih'].apply(convert_to_year_month_day)
X_test['tarih'] = X_test['tarih'].apply(lambda x: generate_value(x))
print(X_test.info())

unique_values_count = X_train['ilce'].nunique()

print("Farklı değerlerin sayısı:", unique_values_count)

# Parametre aralıklarının belirlenmesi
param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200]
}
# Model oluşturma
xgb = XGBRegressor()

# GridSearchCV ile en iyi parametre kombinasyonunun bulunması
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# En iyi parametrelerin bulunması
best_params = grid_search.best_params_
print("En iyi parametreler:", best_params)

# XGBoost modelini eğitme
catboost_model = CatBoostRegressor(**best_params)
catboost_model.fit(X_train, y_train)
# XGBoost modelini eğitme
xgboost_model = XGBRegressor(**best_params)
xgboost_model.fit(X_train, y_train)

# XGBoost modeli ile tahminler
xgboost_preds = xgboost_model.predict(X_test)

# CatBoost modeli ile tahminler
catboost_preds = catboost_model.predict(X_test)

# Tahminlerin ortalamasını alarak ensemble tahmini yapma
ensemble_preds = (xgboost_preds + catboost_preds) / 2

# Sample submission dosyasına tahminleri ekleyerek yeni bir dosya oluşturma
ensemble_submission = sample_submission.copy()
ensemble_submission["bildirimsiz_sum"] = ensemble_preds
ensemble_submission.to_csv("ensemble_submission_1.csv", index=False)





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155552 entries, 0 to 1155551
Data columns (total 14 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   tarih                    1155552 non-null  int64  
 1   ilce                     1155552 non-null  object 
 2   bildirimsiz_sum          1155552 non-null  int64  
 3   bildirimli_sum           1155552 non-null  int64  
 4   lat                      1155552 non-null  float64
 5   lon                      1155552 non-null  float64
 6   t_2m:C                   1155552 non-null  float64
 7   effective_cloud_cover:p  1155552 non-null  float64
 8   global_rad:W             1155552 non-null  float64
 9   relative_humidity_2m:p   1155552 non-null  float64
 10  wind_dir_10m:d           1155552 non-null  float64
 11  wind_speed_10m:ms        1155552 non-null  float64
 12  prob_precip_1h:p         1155552 non-null  float64
 13  t_apparent:C             1155552 non-null 

C:\Users\Yasemin\AppData\Local\Temp\ipykernel_24256\1607508738.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['ilce'] = label_encoder.fit_transform(X_train['ilce'])
C:\Users\Yasemin\AppData\Local\Temp\ipykernel_24256\1607508738.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['ilce'] = label_encoder.fit_transform(X_test['ilce'])
C:\Users\Yasemin\AppData\Local\Temp\ipykernel_24256\1607508738.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

En iyi parametreler: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
0:	learn: 5.4226648	total: 29.8ms	remaining: 1.46s
1:	learn: 5.4197977	total: 61.4ms	remaining: 1.47s
2:	learn: 5.4169862	total: 91.4ms	remaining: 1.43s
3:	learn: 5.4142292	total: 121ms	remaining: 1.39s
4:	learn: 5.4115257	total: 149ms	remaining: 1.34s
5:	learn: 5.4088747	total: 175ms	remaining: 1.28s
6:	learn: 5.4062752	total: 202ms	remaining: 1.24s
7:	learn: 5.4037262	total: 229ms	remaining: 1.2s
8:	learn: 5.4012268	total: 256ms	remaining: 1.17s
9:	learn: 5.3987759	total: 282ms	remaining: 1.13s
10:	learn: 5.3963728	total: 309ms	remaining: 1.1s
11:	learn: 5.3940484	total: 336ms	remaining: 1.06s
12:	learn: 5.3917202	total: 361ms	remaining: 1.03s
13:	learn: 5.3894373	total: 388ms	remaining: 998ms
14:	learn: 5.3871989	total: 417ms	remaining: 973ms
15:	learn: 5.3850010	total: 444ms	remaining: 944ms
16:	learn: 5.3828323	total: 471ms	remaining: 914ms
17:	learn: 5.3807059	total: 498ms	remaining: 886ms
18:	learn: